<a href="https://colab.research.google.com/github/vbloise3/Data_Science/blob/master/GettingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
import sys, re
!pip install -U -q PyDrive
!pip install -U -q beautifulsoup4 requests~=2.21.0 html5lib folium==0.2.1 python-dateutil imgaug<0.2.7,>=0.2.5 twython 

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
from collections import Counter 
from dateutil.parser import parse 
from bs4 import BeautifulSoup 
import requests 
from typing import Dict , Set 
import json 

drive.mount('/content/drive')
with open('/content/drive/My Drive/data.txt', 'w') as f:
  f.write('# Hello Google Drive!')
!cat /content/drive/My\ Drive/data.txt
with open('/content/drive/My Drive/data/data.txt', 'w') as f:
  f.write('\n# Hello Google Drive 1!')
  f.write('\n# Hello Google Drive 2!')
  f.write('\n# Hello Google Drive 3!')
with open('/content/drive/My Drive/data/data.txt', 'r') as fr:
  #print(f"\ndata.txt contents: {fr.read()}")
  starts_with_hash = 0
  for line in fr : 
    # look at each line in the file 
    if re . match ( "^#" , line ): # use a regex to see if it starts with '#' 
      starts_with_hash += 1 # if it does, add 1 to the count 
  print(f"\ncount of comment lines in file: {starts_with_hash}\n")

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1YcA4bNO16W0ZQhn68fs5pWwWsGgZohuv'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content from data.txt "{}"'.format(downloaded.GetContentString()))


# list files 
listed_txt = drive.ListFile({'q': "title contains '.txt' and 'root' in parents"}).GetList()
listed_ipynb = drive.ListFile({'q': "'1C9uuMFnTnllNbyaUWhsHk-PK0bDMpMuQ' in parents"}).GetList()
for file in listed_txt:
  print('title {}. id {}'.format(file['title'], file['id']))
for file in listed_ipynb:
  print('title {}. id {}'.format(file['title'], file['id']))

# sys.argv is the list of command-line arguments 
# sys.argv[ 0] is the name of the program itself 
# sys.argv[ 1] will be the regex specified at the command line 
# run it like this: cat SomeFile.txt | python egrep.py "[ 0-9]" | python line_count.py
regex = sys.argv[ 1]
some_lines = ["one line ", "two lines ", "three! "]

# for every line passed into the script 
for line in sys.stdin: 
  # if it matches the regex, write it to stdout 
  if re.search( regex, line): 
    sys.stdout.write( line)
for line in some_lines:
  sys.stdout.write(line + "\n")

# email address domain stripper example
def get_domain( email_address: str)-> str: 
  """ Split on '@' and return the last piece""" 
  return email_address.lower(). split("@")[-1]

# a couple of tests 
assert get_domain(' joelgrus@gmail.com') == 'gmail.com'
assert get_domain(' joel@m.datasciencester.com') == 'm.datasciencester.com'

domain_counts = 0
with open('/content/drive/My Drive/data/emailAddresses1.txt', 'r') as fa: 
  domain_counts = Counter( get_domain( line.strip()) 
  for line in fa if "@" in line)
print(f"domain count: {domain_counts}\n")

def process(date: str, symbol: str, closing_price: str)-> str:
  print(f"date: {date}, ticker: {symbol}, closing price: {closing_price}\n")
  return symbol

import csv 
ticker_dict = {}
with open('/content/drive/My Drive/data/TickerPrices.txt', 'r') as ftickers: 
  colon_reader = csv.DictReader( ftickers, delimiter =':') 
  for dict_row in colon_reader: 
    ticker_dict.update(dict_row)
    date = dict_row[ "date"] 
    symbol = dict_row[ "symbol"] 
    closing_price = float( dict_row[ "closing_price"]) 
    process( date, symbol, closing_price)

# You can similarly write out delimited data using csv.writer : 
todays_prices = { 'AAPL' : 90.91 , 'MSFT' : 41.68 , 'FB' : 64.5 } 
print(f"dict items: {ticker_dict.items()}\n")
with open ( '/content/drive/My Drive/data/comma_delimited_stock_prices.txt' , 'w' ) as fw: 
  csv_writer = csv.writer ( fw , delimiter = ',' ) 
  for row in ticker_dict.items(): 
    csv_writer.writerow ([ row ]) 
    print(f"item: {row}\n")

# scrape some HTML
url = ( "https://raw.githubusercontent.com/joelgrus/data/master/getting-data.html" ) 
html = requests.get( url ).text 
soup = BeautifulSoup( html , 'html5lib' ) 
print(f"first p-tag of getting-data.html: {soup.p}\n")
print(f"first paragraph text of getting-data.html: {soup.p.text}\n")
print(f"first words in first paragraph of getting-data.html: {soup.p.text.split()}\n")

# scaping the house.gov site
url = "https://www.house.gov/representatives" 
text = requests.get( url ).text 
soup = BeautifulSoup( text , "html5lib" ) 
all_urls = [ a['href'] 
            for a in soup( 'a' ) 
            if a.has_attr( 'href' )] 
print ( len ( all_urls )) # 965 for me, way too many 
# Must start with http:// or https:// 
# Must end with .house.gov or .house.gov/ 
regex = r"^https?://.*\.house\.gov/?$" 
# Let's write some tests! 
assert re.match( regex , "http://joel.house.gov" ) 
assert re.match( regex , "https://joel.house.gov" ) 
assert re.match( regex , "http://joel.house.gov/" ) 
assert re.match( regex , "https://joel.house.gov/" ) 
assert not re.match( regex , "joel.house.gov" ) 
assert not re.match( regex , "http://joel.house.com" ) 
assert not re.match( regex , "https://joel.house.gov/biography" ) # And now apply 
good_urls = [url for url in all_urls if 
             re.match( regex , url )] 
print ( len ( good_urls )) # still 862 for me 
good_urls = list(set( good_urls )) 
print( len ( good_urls )) # only 431 for me 
html = requests.get( 'https://jayapal.house.gov' ).text 
soup = BeautifulSoup( html , 'html5lib' ) # Use a set because the links might appear multiple times. 
links = {a[ 'href' ] for a in soup( 'a' ) if 'press releases' in a.text.lower()} 
print( links ) # {'/media/press-releases'} 


press_releases: Dict[ str , Set [ str ]] = {}
#for house_url in good_urls: 
#  html = requests.get( house_url ).text 
#  soup = BeautifulSoup( html , 'html5lib' ) 
#  pr_links = {a[ 'href' ] for a in soup( 'a' ) if 'press releases' in a.text.lower()} 
#  print(f"{house_url}: {pr_links}") 
#  press_releases[house_url ] = pr_links 

serialized = """{ "title" : "Data Science Book", "author" : "Joel Grus", "publicationYear" : 2019, "topics" : [ "data", "science", "data science"] }""" 
# parse the JSON to create a Python dict 
deserialized = json.loads( serialized ) 
assert deserialized["publicationYear" ] == 2019 
assert "data science" in deserialized[ "topics" ] 


github_user = "vbloise3" 
endpoint = f"https://api.github.com/users/{github_user}/repos" 
repos = json.loads( requests.get( endpoint ).text ) 
dates = [parse(repo [ "created_at" ]) for repo in repos ] 
month_counts = Counter(date.month for date in dates ) 
weekday_counts = Counter ( date.weekday() for date in dates ) 
last_5_repositories = sorted( repos , key = lambda r: r[ "pushed_at" ], reverse = True )[:5] 
last_5_languages = [ repo [ "language" ] for repo in last_5_repositories ] 
print(f"last 5 repo languages: {last_5_languages}\n")
print(f"last 5 repos created: {last_5_repositories}\n")

/bin/bash: 0.2.7,: No such file or directory
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
# Hello Google Drive!
count of comment lines in file: 3

Downloaded content from data.txt "
# Hello Google Drive 1!
# Hello Google Drive 2!
# Hello Google Drive 3!"
title data.txt. id 1yIcwI0LJVpw8rAel9DA8qcJeEfjllkun
title GettingData.ipynb. id 1nKZv3XGsSPtW1WfrirQKRgWQ14s8mp8m
title GradientDescent.ipynb. id 1OL130LGnKNLk9BbzbRekiaTOVBNPc903
title PythonWorkoutChapter1.ipynb. id 1hQ_qWfn_tj8KBQspsqsIJQh8j_dqFq1e
title HypothesisandInference.ipynb. id 1GngAYwLNtVi9kntFYbivstv_XoY6pP8q
title Probability.ipynb. id 1SF6aug-J0inOZTWytKpr6F7s9Bf3nGO2
title Statistics.ipynb. id 1FgvOUIxRhmt5J3KTj_CkTjEkx7W4Pvdz
title Copy of Linear Algebra.ipynb. id 1nbLcny5T4AgdjB6fFa0oVrEq8UVx7u9r
title Linear Algebra.ipynb. id 1az-uba8FteC7F1VbBXPCJYUHvUySKjLd
title firstNN.ipynb. id 1m5fbJPk_nUkXUU1wzV5MmPBzX0Ztum75
title testNN.ipy